##Task 1.1 Data Preparation

In [7]:
import pandas as pd

#read dataset
df = pd.read_csv("glass_dataset.csv")
print(df.head(5)) #print first 5 rows to verify
df.info() 
print("\n")

#check for duplicates 
print("Duplicate count : ",df.duplicated().sum(), "\n") #print duplicate count
print(df[df.duplicated(keep=False)], "\n") #print duplicate rows 

df = df.drop_duplicates() #drop duplicates and reassign df
print("Duplicant count : ",df.duplicated().sum(), "\n")

#check for missing values
print(df.isnull().sum())
print("Missing count: ",df.isnull().sum().sum(), "\n")

        RI     Na    Mg    Al     Si     K    Ca   Ba   Fe  Type
0  1.52101  13.64  4.49  1.10  71.78  0.06  8.75  0.0  0.0     1
1  1.51761  13.89  3.60  1.36  72.73  0.48  7.83  0.0  0.0     1
2  1.51618  13.53  3.55  1.54  72.99  0.39  7.78  0.0  0.0     1
3  1.51766  13.21  3.69  1.29  72.61  0.57  8.22  0.0  0.0     1
4  1.51742  13.27  3.62  1.24  73.08  0.55  8.07  0.0  0.0     1
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214 entries, 0 to 213
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   RI      214 non-null    float64
 1   Na      214 non-null    float64
 2   Mg      214 non-null    float64
 3   Al      214 non-null    float64
 4   Si      214 non-null    float64
 5   K       214 non-null    float64
 6   Ca      214 non-null    float64
 7   Ba      214 non-null    float64
 8   Fe      214 non-null    float64
 9   Type    214 non-null    int64  
dtypes: float64(9), int64(1)
memory usage: 16.8 KB


Duplicat